# Build a model with it's own embedding layer
The prepared clean data can now be used to build a language model. The model will have the following characteristics:
- It uses a distributed respresentation for words so that words with similar meanings have a similar representation.
- It leans the representation at the time as the learning model.
- It learns to predict the probabability for the next word using the context of the last 100 words.

Specifically, I will use an Embedding Layer to learn the representation of words, and a Long Short-Term Memory (LSTM) recurrent neural network to learn to predict words based on their context.

In [1]:
DATASETS_DIR = "../datasets"
MODELS_DIR = "../models"

In [2]:
# load doc into memory
def load_doc(filename):
  # open the file as read only
  file = open(filename, 'r')

  # read all the text
  text = file.read()

  # close the file
  file.close()
  return text

in_filename = f"{DATASETS_DIR}/content_sequences.txt"
doc = load_doc(in_filename)
lines = doc.split('\n')
lines[:4]

['premier soccer league psl iri kudya magaka mambishi zvichitevera mhirizhonga yakaitika',
 'soccer league psl iri kudya magaka mambishi zvichitevera mhirizhonga yakaitika kubabourfields',
 'league psl iri kudya magaka mambishi zvichitevera mhirizhonga yakaitika kubabourfields kubulawayo',
 'psl iri kudya magaka mambishi zvichitevera mhirizhonga yakaitika kubabourfields kubulawayo nezuro']

## Encode sequences with using `Tokenizer`

In [3]:
from pickle import load

# load the tokenizer
tokenizer = load(open(f'{MODELS_DIR}/tokenizer.pkl', 'rb'))

In [4]:
# integer encode sequences of words
sequences = tokenizer.texts_to_sequences(lines)

## Sequence inputs and output
Now that there are encoded input sequences, they need to be seperated into input($X$) and output($y$). This can be done via array slicing.

In [5]:
# Seperate into input and output
from numpy import array
sequences = array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]

In [6]:
X.shape

(5498, 10)

In [7]:
y.shape

(5498,)

After seperating, each output word needs to be one hot encoded. This means converting it from an integer to a vector of 0 values, one for each word in the vocabulary, with a 1 to indicate the specific word at the of the words integer value. This is so that the model learns to predict the probability distribution for the next word and the ground truth from which to learn from is 0 for all words except the actual word that comes next.

In [8]:
# one-hot encode the output word
from keras.utils import to_categorical

vocab_size = len(tokenizer.word_index) + 1

y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
seq_length

10

## Fit the model: Inial training

In [9]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Bidirectional, Dropout
from keras.layers import Embedding
from keras.utils import plot_model

# define the model
def define_model_with_embedding(vocab_size, seq_length, summary=True):
  model = Sequential()
  model.add(Embedding(vocab_size, 100, input_length=seq_length))
  model.add(Bidirectional(LSTM(100)))
  model.add(Dropout(0.2))
  model.add(Dense(100, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(vocab_size, activation='softmax'))

  # compile network
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  # summarize the model
  if summary:
    model.summary()
    model_img_path = f'{MODELS_DIR}/model_with_embedding.png'
    plot_model(model, to_file=model_img_path, show_shapes=True)
  return model

In [10]:
# define model
model_with_embedding = define_model_with_embedding(vocab_size, seq_length)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 100)           276600    
                                                                 
 bidirectional (Bidirection  (None, 200)               160800    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 100)               20100     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 2766)              279366    
                                                        

In [11]:
# fit model
model_with_embedding.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
43/43 [==============================] - 8s 45ms/step - loss: 7.8645 - accuracy: 0.0191
Epoch 2/100
43/43 [==============================] - 2s 44ms/step - loss: 7.4989 - accuracy: 0.0238
Epoch 3/100
43/43 [==============================] - 2s 44ms/step - loss: 7.3511 - accuracy: 0.0238
Epoch 4/100
43/43 [==============================] - 2s 43ms/step - loss: 7.2098 - accuracy: 0.0242
Epoch 5/100
43/43 [==============================] - 2s 42ms/step - loss: 7.0139 - accuracy: 0.0246
Epoch 6/100
43/43 [==============================] - 2s 44ms/step - loss: 6.8372 - accuracy: 0.0233
Epoch 7/100
43/43 [==============================] - 2s 44ms/step - loss: 6.6735 - accuracy: 0.0247
Epoch 8/100
43/43 [==============================] - 2s 42ms/step - loss: 6.5169 - accuracy: 0.0247
Epoch 9/100
43/43 [==============================] - 2s 45ms/step - loss: 6.3926 - accuracy: 0.0264
Epoch 10/100
43/43 [==============================] - 2s 43ms/step - loss: 6.2934 - accuracy: 0.0258

In [22]:
from pickle import dump

# save the model to file
model_with_embedding.save(f'{MODELS_DIR}/model_with_embedding.keras', save_format="tf")

In [13]:
# Make new predictions
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import numpy as np
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
  result = list()
  in_text = seed_text
  # generate a fixed number of words
  for _ in range(n_words):
    # encode the text as integer
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    # truncate sequences to a fixed length
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    # predict probabilities for each word
    yhat = model.predict(encoded, verbose=0)
    # map predicted word index to word
    predicted_word_index = np.argmax(yhat)  # Find the index with the highest probability
    out_word = ''
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            out_word = word
            break
    # append to input
    in_text += ' ' + out_word
    result.append(out_word)
  return ' '.join(result)

In [ ]:
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

mapurisa vakaona kuti akauraiwa uyu ndinato chikore wekumkoba uyo mutumbi wake



In [21]:
seed_text = "kuti nhavu yemuno iyende mberi zvakakosha kuti vagari vemuno vange"
pred = generate_seq(model_with_embedding, tokenizer, seq_length, seed_text, 10)
print(seed_text + ' [START] ' + pred + ' [END]')

kuti nhavu yemuno iyende mberi zvakakosha kuti vagari vemuno vange [START] hainei zviye nesu pfungwa kwemhepo hwababa anoti zvekuibvisa munhu joseph [END]


## Training with cross validation for model selection

In [42]:
from sklearn.model_selection import KFold
import numpy as np

In [38]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [ ]:
# define 5-fold cross validation test harness
kfold = KFold(n_splits=5, shuffle=True, random_state=seed)

cvscores = []
for train, test in kfold.split(X, y):
    # compile the model
    model_with_embedding = define_model_with_embedding(vocab_size, seq_length, summary=False)
    
    # Fit the model
    model_with_embedding.fit(X[train], y[train], epochs=20, batch_size=128)
    
    # Evaluate the model
    model_with_embedding.evaluate(X[test], y[test])
    print("Accuracy = %s: %.2f%%" % (model_with_embedding.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("Mean Accuracy = %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))